<a href="https://colab.research.google.com/github/biini/csproject1/blob/main/AI_06_%EB%B0%95%EC%9B%90%EB%B9%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## KoBART모델을 활용한 텍스트 요약
> 에러를 해결하지 못해 완성하지 못했다.

In [ ]:
!pip install datasets

In [ ]:
!pip install rouge_score

In [ ]:
!pip install transformers

In [12]:
## 라이브러리

import pandas as pd 

import os, json
from pandas.io.json import json_normalize
import glob

from datasets import load_metric
from datasets import Dataset

import torch, gc
from torch.utils.data import DataLoader

from transformers import PreTrainedTokenizerFast,BartTokenizer
from transformers import BartForConditionalGeneration
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AdamW

In [5]:
metric = load_metric("rouge")

## 데이터 형태 만들기

In [6]:
## 데이터 불러오기
dir_train = '/content/drive/MyDrive/Colab Notebooks/Project/1/data/training_특허전체/' #파일경로

json_files = [files for files in os.listdir(dir_train)] # 파일안에 있는 모든 파일들의 이름을 리스트에 담기

data_list_train = []
for f in json_files:
  with open(dir_train + f) as data: # 경로에 있는 파일들 하나씩 열기
    json_data = json.load(data) # 파일을 파이썬 객체로 직렬화
    data = json_normalize(json_data['data'], 'summary_entire') # json_data에서 'data'부분을 불러와 그 안에있는 'summary_entire'만 데이터 프레임으로 만들기

    data_list_train.append(data) # 정보들을 데이터 프레임 형태로 data_list에 하나씩 넣기

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.


In [7]:
# 데이터프레임 합치기
df_train = pd.concat(data_list_train, axis=0, ignore_index=True)
print('len: ', len(df_train))

len:  17800


In [8]:
# 데이터 줄이기(데이터가 너무 많아 tokenize가 안됨)
df_train = df_train.iloc[:500]
# dataset으로 알맞은 데이터 형태 만들기
df = pd.DataFrame({"original_text": df_train['orginal_text'],
                   "summary_text": df_train['summary_text']})
dataset = Dataset.from_pandas(df)

In [9]:
dataset

Dataset({
    features: ['original_text', 'summary_text'],
    num_rows: 500
})

In [10]:
# 데이터셋 나누기
dataset = dataset.train_test_split(test_size=0.2)

## fine-tuning1
- hugging face의 transformers라이브러리 이용(`trainer`)

### tokenize

In [14]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [15]:
max_input_length = 1024
max_target_length = 128

# 토큰화 함수
def preprocess_function(examples):
    # 원본 데이터 토큰화
    inputs = [doc for doc in examples["original_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟 데이터에 대한 토큰화
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [16]:
# 토큰화 확인
preprocess_function(dataset['train'][:2])

{'input_ids': [[14095, 290, 14103, 14916, 21351, 1700, 19532, 11810, 12126, 15247, 14069, 9983, 283, 271, 27271, 276, 278, 268, 275, 27628, 283, 281, 21292, 15464, 268, 277, 27271, 264, 283, 272, 278, 277, 14759, 278, 267, 284, 275, 268, 230, 290, 14702, 18307, 1700, 19532, 10888, 14103, 16237, 14189, 12126, 14296, 13716, 14081, 14631, 14533, 16567, 14488, 9866, 15828, 27302, 19749, 14211, 14049, 14189, 12126, 15247, 14069, 9983, 11786, 15501, 16746, 14095, 290, 17194, 15131, 1700, 19532, 12191, 10232, 14150, 243, 14189, 12126, 14296, 13716, 14081, 14631, 14068, 11843, 12846, 14189, 9160, 13716, 16242, 14209, 11843, 12846, 14189, 9160, 13716, 9264, 16711, 17217, 14330, 19749, 14211, 14049, 14189, 12126, 15247, 12034, 18461, 15964, 14189, 12126, 14296, 13716, 14081, 14259, 14060, 10850, 14980, 14218, 18783, 14189, 12126, 17325, 28276, 13590, 16746, 14189, 12126, 22317, 15391, 20097, 14154, 15078, 21229, 17155, 243, 15073, 21162, 20097, 14189, 12126, 22317, 10338, 16615, 9908, 14189, 121

In [17]:
# 토큰화
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
# 모델 선언
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

In [19]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [20]:
gc.collect()
torch.cuda.empty_cache()

In [21]:
batch_size = 8

args = Seq2SeqTrainingArguments(
    output_dir = "/content/drive/MyDrive/Colab Notebooks/Project/1",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary_text, original_text, token_type_ids.
***** Running training *****
  Num examples = 400
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 50


RuntimeError: ignored

## finetuing2
- pytorch 활용

In [25]:
max_input_length = 1024
max_target_length = 128

# 토큰화 함수
def preprocess_function(examples):
    # 원본 데이터 토큰화
    inputs = [doc for doc in examples["original_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # 타겟 데이터에 대한 토큰화
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary_text"], max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [26]:
# 토큰화
tokenized_datasets2 = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
model.to(device)
model.train()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

In [31]:
train_loader = DataLoader(tokenized_datasets2['train'], batch_size=16, shuffle=True)
optim = AdamW(model.parameters(),lr=5e-5)

In [32]:
num_train_epochs = 2
for epoch in range(num_train_epochs):
  for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

    loss = outputs[0]
    loss.backward()
    optim.step()

model.eval

AttributeError: ignored

## 참고 자료

- [텍스트 요약에 대한 전반적 설명](https://github.com/uoneway/Text-Summarization-Repo)
- [huggingface 공식문서](https://huggingface.co/docs/transformers/training)
- [huggingface 공식문서](https://github.com/huggingface/notebooks/blob/master/examples/summarization.ipynb)
- [텍스트 요약 프로젝트 영상](https://www.youtube.com/watch?v=XG-weJ06bOQ)